# Seattle AirBnB dataset analysis

Seattle is a wonderful to visit, and AirBnB is a wonderful way to find a place to stay while visiting any place. Let's see what kind of insight I can gather for my next trip to Seattle from AirBnB's Seattle data.


## Procedures

* Explore the dataset
* Pose a few questions
* Clean the data to get ready for analysis
* Analyze the data to answer the questions
* Present graphics where needed

In [1]:
# import relevant packages
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import seaborn as sns #unused
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

#set up data path
#this is currently the path to access the data on kaggle, modify as needed
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
#read in data
list_df=pd.read_csv('/kaggle/input/seattle/listings.csv')
review_df=pd.read_csv('/kaggle/input/seattle/reviews.csv')
date_df=pd.read_csv('/kaggle/input/seattle/calendar.csv')


### Explore Dataframes

First check the listings dataframe

In [3]:
print(list_df.shape)
list_df.head()

The listing dataset has 92 columns that describe 3818 property listings

In [4]:
#what variables can we play with
list_df.columns

In [5]:
#quick summary of data
list_df.describe()

In [6]:
#get an understanding of categorical and numerical data columns
list_df.select_dtypes('number').columns

In [7]:
list_df.describe(include='object')

Next, check the dataframe containing reviews

In [8]:
print(review_df.shape)
review_df.head()

almost 85000 comments on the listings in the previous dataset

In [9]:
review_df.describe(include='object')

Third, review the calendar dataframe

In [10]:
print(date_df.shape)
date_df.head()

In [11]:
date_df.describe(include='object')

In [12]:
print('first date in the dataset is '+date_df['date'].min()+' and last date is '+date_df['date'].max())

The calendar dataset contains when the properties are available and their pricing. Some prices are missing. They appear to be if the unit is unavailable Available is boolean but uses t,f instead. It looks like all of the data ares from one year, roughly the year of 2016

## What Problems am I interested in?
Based on my EDA, there are 3 questions that would be useful or interesting to me for my next trip to Seattle
1. What is the cheapest time to lodge in Seattle?
2. Where is the cheapest area to stay?
3. What are the features that most impact rental price?

Before answering the questions, the datasets have to be cleaned-up first

## Data Cleaning

First clean the listing dataframe

In [13]:
#which columns are not missing data
len(list_df.columns[list_df.isnull().sum()==0])

In [14]:
#how much data is missing per column
list_df[list_df.columns[list_df.isnull().sum()>0]].isnull().mean().sort_values()

Around 10 columns should be dropped due to too much missing data

Also, there is no missing data for price, our target variable


For finding the cheapest areas, I will need to group the data by neighbourhoods. Let's check if there is enough data for each neighbourhood

In [15]:
list_df['neighbourhood_cleansed'].value_counts().hist(bins=40)

some neighbourhoods don't have enough data, maybe that's why they also provided a "neighbourhood groups" column

In [16]:
list_df['neighbourhood_group_cleansed'].value_counts()

It is better to analyze the neighbourhood groups, since there is much more data per group

In [17]:
#some columns have strange formatting and hey need to be fixed
def pricing_reformat(df_col):
    return df_col.str.strip('$').str.replace(',','').astype(float)

list_df['price']=pricing_reformat(list_df['price'])
list_df['extra_people']=pricing_reformat(list_df['extra_people'])

Check missing data for the reviews dataframe. I won't be using this dataframe going forward, so no cleaning is done

In [18]:
review_df.isnull().sum()

Now, clean the calendar data

In [19]:
# rows missing prices are not useful since I'm interested in pricing, so drop them
date_df2=date_df.dropna(subset=['price'])
date_df2.describe(include='object')

In [20]:
# as expected, all rows with prices are for when the listing is available, so the avalability column isn't very useful
date_df2=date_df2.drop(['available'],axis=1)
date_df2.head()

In [21]:
# I want to analyze pricing by month and by date, create relevant columns
y_md=date_df2['date'].str.split('-',expand=True,n=1)
date_df2=date_df2.assign(monthday=y_md[1], month=y_md[1].str.split('-',expand=True)[0].astype(int))
date_df2['price']=pricing_reformat(date_df2['price'])

date_df2.head()

In [22]:
#need to check there are enough data for each month and each date
date_df2['monthday'].hist(bins=365,xrot=90)

OK there should be enough data points per date to make reasonable conclusions

For modeling to answer my 3rd question, it is best to increase the number of data points. To do this, I can merge list_df and model_df based on their listing IDs

In [23]:
list_df=list_df.rename(index=str, columns={'id':'listing_id'})
model_df = pd.merge(date_df2, list_df, on = 'listing_id')
model_df.shape

We get significantly more data to work with

In [24]:
### drop columns

#these cols have way too many missing numbers to be predictive
drop_cols=['cleaning_fee','neighborhood_overview','notes','weekly_price','security_deposit','monthly_price','square_feet','license']
model_df=model_df.drop(drop_cols, axis=1)

#to reduce dimensionality of the problem, I decided to drop the following columns
drop_cols = ['listing_id', 'monthday','date', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name','host_since', 'host_response_rate',
       'host_location', 'host_about', 'host_response_time','city','state',
        'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
        'zipcode', 'market', 'transit',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact',  'room_type', 'bed_type', 'amenities', 'minimum_nights',
       'maximum_nights', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped',
       'first_review', 'last_review',  'requires_license',
        'jurisdiction_names', 'instant_bookable',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification', 'calculated_host_listings_count',
       'reviews_per_month','price_y']
model_df=model_df.drop(drop_cols, axis=1)

In [25]:
#drop any row with missing price
model_df.dropna(subset=['price_x'])

#split response column
y=model_df['price_x']
model_df=model_df.drop(['price_x'],axis=1)

#fill missing vals for numerical variables
num_cols=model_df.select_dtypes(include=['float','int']).columns
for col in num_cols:
    model_df[col].fillna(model_df[col].median(), inplace=True)

#dummy variables for categorical variables
cat_cols=model_df.select_dtypes(include=['object']).columns
for col in cat_cols:
    model_df=pd.concat([model_df.drop(col, axis=1), pd.get_dummies(model_df[col], prefix=col, prefix_sep='_', drop_first=True)], axis=1)

model_df.head()

# Analysis
## Q1: Cheapest Time to visit Seattle

Approach:
* group by month and calculate average price
* group by date and calculate average price
* plot the results

The results will show the cheapest month and dates

In [26]:
#calculate monthly and daily price averages
monthly_prices=date_df2.groupby(['month'], as_index=False, group_keys=False)['price']
daily_prices=date_df2.groupby(['monthday'], as_index=False, group_keys=False)['price']
monthly_avg=monthly_prices.mean()
daily_avg=daily_prices.mean()
daily_avg

In [27]:
%matplotlib inline
plt.plot(monthly_avg['month'],monthly_avg['price'])
plt.xlabel('month')
plt.ylabel('prices per day')
plt.title('average price by month')



In [28]:
print(f'the cheapest month is January with average price of ${monthly_avg.price.min():.2f}')
print(f'the most expensive month is July with average price of ${monthly_avg.price.max():.2f}')

In [29]:

fig, ax = plt.subplots()
ax.plot(daily_avg['monthday'],daily_avg['price'])
plt.xlabel('date')
plt.xticks(rotation=90, fontsize=8)
loc = plticker.MultipleLocator(base=14.0) 
ax.xaxis.set_major_locator(loc)
plt.ylabel('prices per day')
plt.title('average price by date')

In [30]:
max_val=daily_avg['price'].max()
max_date=daily_avg.iloc[daily_avg['price'].argmax()].monthday
min_val=daily_avg['price'].min()
min_date=daily_avg.iloc[daily_avg['price'].argmin()].monthday
print(f'maximum daily average prices is ${max_val:.2f} on {max_date}')
print(f'minimum daily average prices is ${min_val:.2f} on {min_date}')

There are spikes in the prices is due to the weekends. I want to get a better sense of how prices change week to week. To do so, I apply a rolling average of 7 days to smooth things over.

In [31]:
daily_avg_smooth=daily_avg.rolling(7, center=True).mean()  
daily_avg_smooth.head()

We are loosing a bit of data at the beginning and end of the year, but they shouldn't affect our results too much

In [32]:
fig, ax = plt.subplots()
ax.plot(daily_avg['monthday'],daily_avg_smooth['price'])
plt.xlabel('date')
plt.xticks(rotation=90, fontsize=8)
loc = plticker.MultipleLocator(base=14.0) 
ax.xaxis.set_major_locator(loc)
plt.ylabel('prices per day')
plt.title('average price by date')


In [33]:
max_val=daily_avg_smooth['price'].max()
max_date=daily_avg.iloc[daily_avg_smooth['price'].argmax()].monthday
min_val=daily_avg_smooth['price'].min()
min_date=daily_avg.iloc[daily_avg_smooth['price'].argmin()].monthday
print(f'maximum daily average prices is ${max_val:.2f} in the week +/-3 days around {max_date}')
print(f'minimum daily average prices is ${min_val:.2f} in the week +/-3 days around {min_date}')

Peak prices are actually centered around July 4th (Independence Day in Seattle), especially since in 2016 July 4th was on a Monday, making it a long weekend that many people would want to travel. On the other hand, late January is when prices fall to its minimum

## Question 2: cheapest area

Approach
* Group listings by neighbourhoods and find average price for each area

The results will show which areas are the cheapest to stay

In [34]:
area_prices=list_df.groupby(['neighbourhood_group_cleansed'], as_index=False, group_keys=False)['price']
area_avg=area_prices.mean().sort_values(['price'])
area_avg

In [35]:
plt.bar(area_avg['neighbourhood_group_cleansed'],area_avg['price'])
plt.xticks(rotation=90)
plt.xlabel('Neighbourhood')
plt.ylabel('prices per day')
plt.title('Average price by area')

This makes it clear the cheapest places to lodge is Delridge, followed closely by Northgate, Rainier Valley, Lake City, University District, and Beacon Hill

Those most expensive place, and by a significant margin, is Magnolia.

## Q3: Top factors influencing price

Now I want to figure out what are the top factors that affect price.

Approach:
* create a model that can accurately predict prices
* examine the weights for each factor

I used the random forest regression model and the r2 and MSE criteria to evaluation


In [ ]:

## modelling
X_train, X_test, y_train, y_test = train_test_split(model_df, y, test_size = 0.2, random_state=7)
#lin_model = LinearRegression(normalize=True) # Instantiate
rf_model = RandomForestRegressor(n_estimators=150, 
                               criterion='mse', random_state=7, n_jobs=-1)

rf_model.fit(X_train, y_train) #Fit

result_train = rf_model.predict(X_train)
result_test = rf_model.predict(X_test)
    
test_score = r2_score(y_test, result_test)
train_score= r2_score(y_train, result_train)

print('R2 test score: '+str(test_score))
print('R2 train score: '+str(train_score))

In [41]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

X_train, X_test, y_train, y_test = train_test_split(model_df, y, test_size = 0.2, random_state=7)

rf_model = RandomForestRegressor(random_state=7, n_jobs=-1, oob_score=True, verbose=1)




In [48]:
param_grid = {'max_depth': randint(5,30), 
              'max_features': randint(5,30),
              'n_estimators':randint(100,500),
              'min_samples_split':randint(2,5)}

rf_cv =RandomizedSearchCV(rf_model, param_grid, scoring="r2",     # (and our parameter grid) to a new instance
                      n_jobs=-1, verbose=1, n_iter=8, cv=3)

In [49]:
rf_cv.fit(X_train, y_train)
print(rf_cv.best_score_)

In [51]:
print(rf_cv.best_params_)

In [56]:
rf_final = RandomForestRegressor(random_state=7, n_jobs=-1, oob_score=True, max_depth =27, max_features=20, min_samples_split=3, n_estimators=308)

rf_final.fit(X_train, y_train)

result_train = rf_final.predict(X_train)
result_test = rf_final.predict(X_test)

test_score = r2_score(y_test, result_test)
train_score= r2_score(y_train, result_train)

print('R2 test score: '+str(test_score))
print('R2 train score: '+str(train_score))


test_score = mean_squared_error(y_test, result_test)
train_score= mean_squared_error(y_train, result_train)

print('MSE test score: '+str(test_score))
print('MSE train score: '+str(train_score))

In [58]:
headers = ["name", "score"]
values = sorted(zip(X_train.columns, rf_final.feature_importances_), key=lambda x: x[1] * -1)
forest_feature_importances = pd.DataFrame(values, columns = headers)
forest_feature_importances = forest_feature_importances.sort_values(by = ['score'], ascending = False)

features = forest_feature_importances['name'][:20]
y_pos = np.arange(len(features))
scores = forest_feature_importances['score'][:20]

#plot feature importances
plt.figure()
plt.bar(y_pos, scores, align='center', alpha=0.5)
plt.xticks(y_pos, features, rotation='vertical')
plt.ylabel('Score')
plt.xlabel('Features')
plt.title('Feature importances')
 
plt.show()

Not too surprisingly, the most important determinants of price are the size and location of the property. Time of the year and the number of reviews also influences price.

Since number of reviews is a top factor affecting price, one follow-up question is: am I more likely to get a worse rental if I go for one with less reviews?

A data fit should help us to see if there is any correlation

In [59]:
list_df2 = list_df[list_df['number_of_reviews']>5] #get rid of extremes that might skew data 
plt.figure()
plt.plot(list_df2['number_of_reviews'],list_df2['review_scores_rating'],'o',label='data')
lin_fit_param=np.polyfit(list_df2['number_of_reviews'], list_df2['review_scores_rating'],1)
plt.plot(list_df2['number_of_reviews'], lin_fit_param[0]*list_df2['number_of_reviews']+lin_fit_param[1],0, label='fit line')
plt.xlabel('number of reviews')
plt.ylabel('average score of reviews')
plt.title('Does more reviews mean better reviews?')
plt.legend(loc='lower right')
print(f'slope of linear fit is {lin_fit_param[0]:.5f}')

There is a very weak correlation.

If we group the data by how many reviews they get, we can also see that the difference is very small. I think it is safe to say it is fine to go for rentals with less reviews.

In [60]:
list_df3 = list_df[list_df['number_of_reviews']>=20]  #number of rentals with more than 20 reviews
list_df4 = list_df[list_df['number_of_reviews']<20] 
print(list_df3.shape) #check that the number of data points is roughly the same
print(list_df4.shape)
x=['less than 20','more than or equal 20']
y=[list_df3['review_scores_rating'].mean(),list_df4['review_scores_rating'].mean()]
plt.figure()
plt.bar(x,y)
plt.xlabel('number of reviews')
plt.ylabel('average score')
plt.title('Average score compared with number of reviews')